<a href="https://colab.research.google.com/github/UEPP40/PUM/blob/ostrowska/regresja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
data = pd.read_csv("data.csv")
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,9,1,0,159,7,13
1,5,0,1,17,5,8
2,22,1,2,130,17,25
3,13,0,0,101,9,7
4,29,1,1,22,22,33


In [45]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder


# enkoder płeć, edu level
le = LabelEncoder()
#data.Gender = le.fit_transform(data.Gender)

columns = ["Gender", "Education Level", "Job Title"]
df[column] = le.fit_transform(df[column])

# sprawdź transformacje
df.head()

,Age,Gender,Education Level,Job Title,Years of Experience,Salary
0,9,1,0,159,7,13
1,5,0,1,17,5,8
2,22,1,2,130,17,25
3,13,0,0,101,9,7
4,29,1,1,22,22,33


In [46]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#podział danych
train_df, test_df = train_test_split(df ,test_size=0.3, random_state=40)

X_train = train_df[["Age", "Gender", "Education Level", "Job Title", "Years of Experience"]]
Y_train = train_df["Salary"]
X_test = test_df[["Age", "Gender", "Education Level", "Job Title", "Years of Experience"]]
Y_test = test_df["Salary"]


In [47]:
model=LinearRegression().fit(X_train,Y_train)
model.score(X_train,Y_train)
model.coef_
model.intercept_
Y_pred = model.predict(X_test)

In [48]:
from sklearn.metrics import mean_absolute_error
Y_true = Y_test
mean_absolute_error(Y_true,Y_pred)

2.288498760984042

In [49]:
from sklearn.metrics import median_absolute_error
Y_true = Y_test
median_absolute_error(Y_true,Y_pred)

1.750105914619457

In [50]:
from sklearn.metrics import r2_score
Y_true = Y_test
r2_score(Y_true,Y_pred)

0.8929123439501764